In [121]:
import numpy as np
import plotly as plt
from scipy.optimize import minimize
from tqdm import tqdm

In [122]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

In [123]:
eps = 0.0012   # Per share fee parameter
delta = 0.005   # tick size
gamma = 5 # inventory penelisation 
m = 6  # the number of states of the spread
n = 100 # grid of inventory
e = 100 # max volumne for market orders
T = 300 # timespan
dt = T / 100 # time difference for Euler scheme
l_max = 100 # maximum volume of an ordinary order
initial_l = 0 # initial guess for optimisation inside the differential equation

In [124]:
intensity_matrix_a = np.array([[0.0539, 0.1485], 
                              [0.0465, 0.0979],
                              [0.0401, 0.0846],
                              [0.0360, 0.0856],
                              [0.0435, 0.1009],
                              [0.0554, 0.1202]]) # Estimated intensities for given quotes and spread for ask

intensity_matrix_b = np.array([[0.0718, 0.1763], 
                              [0.0520, 0.1144],
                              [0.0419, 0.0915],
                              [0.0409, 0.0896],
                              [0.0452, 0.0930],
                              [0.0614, 0.1255]])    # Estimated intensities for given quotes and spread for bid

intensity_matrix_sym = (intensity_matrix_a + intensity_matrix_b) / 2    # Making bid and ask sides symmetric

In [125]:
prob_matrix = np.array([[0, 0.41, 0.22, 0.16, 0.142, 0.065], 
                        [0.201, 0, 0.435, 0.192, 0.103, 0.067],
                        [0.113, 0.221, 0, 0.4582, 0.147, 0.059],
                        [0.07, 0.085, 0.275, 0, 0.465, 0.102],
                        [0.068, 0.049, 0.073, 0.363, 0, 0.446],
                        [0.077, 0.057, 0.059, 0.112, 0.692, 0]])   # probability transition matrix
R = prob_matrix
R[np.arange(0, R.shape[0]), np.arange(0, R.shape[0])] = -np.sum(R, axis = 1)  # intensity matrix of Markov chain that determines the spread process

In [126]:
i = 1 # spread position
y = np.linspace(-1000, 1000, n) # mesh grid in inventory

In [127]:
def g(y):
    return y ** 2

In [128]:
def phi_objective(l, tick, i, delta, phi_t, y, y_current, intensity_matrix, bid_or_ask):
    
    lambda_phi = intensity_matrix[i, tick]

    if bid_or_ask == "ask":
        y_shift = y_current - l
    else:
        y_shift = y_current + l

    #y_broadcasted = y.reshape(1, -1)
    #shifted_y_broadcasted = y_shift.reshape(-1, 1)
    #y_difference = np.abs(y_broadcasted - shifted_y_broadcasted)
    #y_index = np.argmin(y_difference, axis = 1)
    y_index_l = np.argmin(np.abs(y - y_shift))
    y_index = np.argmin(np.abs(y - y_current))
    phi_y_l = phi_t[y_index_l]
    phi_t = phi_t[y_index]

    if bid_or_ask == "ask":
        objective = lambda_phi * (phi_y_l - phi_t + (i *  delta / 2 - delta * tick) * l)
    else:
        objective = lambda_phi * (phi_y_l - phi_t + (i *  delta / 2 - delta * tick) * l)
    print(objective)
    return -objective

In [129]:
def optimise_phi(y, y_current, initial_l, phi_t, delta, i, intensity_matrix, bid_or_ask):
    

    solution_tick = minimize(phi_objective, initial_l, bounds = [(0, l_max)], 
                                args = (0, i, delta, phi_t, y, y_current, intensity_matrix, bid_or_ask))

    solution_normal = minimize(phi_objective, initial_l, bounds = [(0, l_max)], 
                                args = (1, i, delta, phi_t, y, y_current, intensity_matrix, bid_or_ask))


    if solution_tick.x[0] > solution_normal.x[0]:
        solution = [solution_tick.x[0], 1.0]
        f = solution_tick.fun[0]
    else:
        solution = [solution_normal.x[0], 1.0]
        f = solution_normal.fun[0]

    return (solution, f)

In [130]:
phi_terminal = -np.abs(y) * i * delta / 2 - eps

In [131]:
phi = [phi_terminal]
q_b = [0]
l_b = [0]
q_a = [0]
l_a = [0]
phi_temp = phi_terminal

for t in tqdm(range(1, int(T / dt))):

    phi_ask_result = []
    phi_bid_result = []
    q_b_temp = []
    l_b_temp = []
    q_a_temp = []
    l_a_temp = []

    for y_current in np.nditer(y):
        phi_opt_ask = optimise_phi(y_current = y_current, y = y, initial_l = initial_l, phi_t = phi_temp, 
                                    delta = delta, i = i, intensity_matrix = intensity_matrix_sym, bid_or_ask = "ask")
        phi_opt_bid = optimise_phi(y_current = y_current, y = y, initial_l = initial_l, phi_t = phi_temp, 
                                    delta = delta, i = i, intensity_matrix = intensity_matrix_sym, bid_or_ask = "bid")
        phi_ask_result.append(phi_opt_ask[1])
        l_b_temp.append(phi_opt_ask[0][0])
        q_b_temp.append(phi_opt_ask[0][1])
        phi_bid_result.append(phi_opt_bid[1])
        l_a_temp.append(phi_opt_bid[0][0])
        q_a_temp.append(phi_opt_bid[0][1])

    phi_temp = -dt * (np.array(phi_ask_result) + np.array(phi_bid_result) - gamma * g(y))
    phi.append(phi_temp)
    q_b.append(q_b)
    l_b.append(l_b_temp)
    q_a.append(q_a_temp)
    l_a.append(l_b)


phi.reverse()

  0%|          | 0/99 [00:00<?, ?it/s]

[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.0123125]
[0.0123125]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.02474937]
[0.02474937]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.00982513]
[0.00982513]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.02474937]
[0.02474937]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.00733775]
[0.00733775]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.02474937]
[0.02474937]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.00485038]
[0.00485038]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.02474937]
[0.02474937]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.00236301]
[0.00236301]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.02474937]
[0.02474937]
[0.]
[-2.65375e-12]
[0.]
[1.2312

  1%|          | 1/99 [00:06<10:07,  6.20s/it]


[0.00733775]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.02474937]
[0.02474937]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.00982513]
[0.00982513]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.02474937]
[0.02474937]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.0123125]
[0.0123125]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[0.0123125]
[0.0123125]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[-141704.91739148]
[-141704.91739148]
[1.53380672e-08]
[1.53392985e-08]
[-58490.96488791]
[-58490.96488791]
[1.54460592e-08]
[1.54472904e-08]
[1.55540511e-08]
[1.55552824e-08]
[-29546.9822909]
[-29546.9822909]
[1.56074961e-08]
[1.56087273e-08]
[1.56609411e-08]
[1.56621723e-08]
[-29546.98382996]
[-29546.98382996]
[1.56743023e-08]
[1.56755336e-08]
[1.56876636e-08]
[1.56888948e-08]
[0.00076955]
[0.00076955]
[-29546.984091

  2%|▏         | 2/99 [00:11<09:15,  5.73s/it]

...

  6%|▌         | 6/99 [00:35<09:04,  5.86s/it]


[0.0012371]
[0.0012371]
[0.0012371]
[0.0012371]
[-24694.38438399]
[-24694.38438399]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[50965.02097369]
[50965.02097369]
[0.]
[-2.65375e-12]
[0.]
[1.23125e-12]
[1.51597656e-08]
[1.51609969e-08]
[-124149.97471357]
[-124149.97471357]
[1.53632793e-08]
[1.53645105e-08]
[-49914.17757989]
[-49914.17757989]
[1.54898275e-08]
[1.54910588e-08]
[1.56163758e-08]
[1.5617607e-08]
[-25219.79502139]
[-25219.79502139]
[1.56789908e-08]
[1.56802221e-08]
[-25219.79606796]
[-25219.79606796]
[1.57062659e-08]
[1.57074972e-08]
[1.5733541e-08]
[1.57347723e-08]
[0.0010158]
[0.0010158]
[0.0010158]
[0.0010158]
[0.0010158]
[0.0010158]
[-25219.79657585]
[-25219.79657585]
[0.0010158]
[0.0010158]
[0.0010158]
[0.0010158]
[-25219.79682979]
[-25219.79682979]
[0.0010158]
[0.0010158]
[0.00101822]
[0.00101822]
[0.00114398]
[0.00114398]
[3.03195312e-08]
[3.03207625e-08]
[9.09585937e-08]
[9.0959825e-08]
[3.33514844e-07]
[3.33516075e-07]
[1.30373984e-06]
[

  7%|▋         | 7/99 [00:41<08:58,  5.86s/it]


[-50960.8841232]
[-50960.8841232]
[1.54883625e-08]
[1.54895937e-08]
[1.56123115e-08]
[1.56135427e-08]
[-25743.1270497]
[-25743.1270497]
[1.56736536e-08]
[1.56748849e-08]
[1.57349957e-08]
[1.5736227e-08]
[-25743.12858876]
[-25743.12858876]
[1.57503313e-08]
[1.57515625e-08]
[0.0010158]
[0.0010158]
[0.0010158]
[0.0010158]
[0.0010158]
[0.0010158]
[-25743.1288504]
[-25743.1288504]
[0.0010158]
[0.0010158]
[0.0010158]
[0.0010158]
[0.00114662]
[0.00114662]
[0.00114662]
[0.00114662]
[0.00114662]
[0.00114662]
[0.00121203]
[0.00121203]
[3.03195312e-08]
[3.03207625e-08]
[9.09585937e-08]
[9.0959825e-08]
[3.33514844e-07]
[3.33516075e-07]
[1.30373984e-06]
[1.30374107e-06]
[5.18463984e-06]
[5.18464107e-06]
[2.07082398e-05]
[2.07082411e-05]
[8.28026398e-05]
[8.28026411e-05]
[0.00033118]
[0.00033118]
[-25743.12880315]
[-25743.12880315]
[0.00033118]
[0.00033118]
[0.00033118]
[0.00033118]
[0.00082794]
[0.00082794]
[0.00082794]
[0.00082794]
[0.00082794]
[0.00082794]
[0.00107631]
[0.00107631]
[0.00124024]


KeyboardInterrupt: 

In [ ]:
0.0539 * (phi_terminal[99] - phi_terminal[0] + i * delta / 2 * 100)

0.013475